# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

# Get data from html

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
import urllib.request

with urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as url:
    Html = url.read()

Soup = BeautifulSoup(Html, 'lxml')
Soup.prettify
table = Soup.find('table', class_ ='wikitable sortable')
list1 = table.find_all('tr')
list2=[]
for i in list1:
    a = i.text.split('\n')[1:-1]
    list2.append(a)
    
list2[0][-1] = 'Neighborhood'   
df1 = pd.DataFrame(list2[1:], columns=list2[0])

df1.head()



,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Data wrangling

In [5]:
not_boroughs = df1.index[df1['Borough'] == 'Not assigned']

df1.drop(df1.index[not_boroughs], inplace=True)
df1.reset_index(drop=True, inplace=True)
not_neighborhood = df1.index[df1['Neighborhood'] == 'Not assigned'] 
for k in not_neighborhood:
    df1['Neighborhood'][k] ==df1['Borough'][k]  
group = df1.groupby('Postcode')                         

list_neighborhoods = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
list_boroughs = group['Borough'].apply(lambda x: set(x).pop())
df2 = pd.DataFrame(list(zip(list_boroughs.index, list_boroughs, list_neighborhoods)))
df2.columns = ['Postcode', 'Borough', 'Neighborhood']
df2.head(20)



,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
print('df2.shape:',df2.shape)

df2.shape: (103, 3)


In [7]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data')
df3 = df2.join(coordinates_df.set_index('Postal Code'), on='Postcode')
df3.head(20)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Generate Map

In [23]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins


Solving environment: done

# All requested packages already installed.



In [24]:
map = folium.Map(location=[43.6532,-79.3832], zoom_start=12)
neighborhood = plugins.MarkerCluster().add_to(map)
for lat, lng, label, in zip(df3.Latitude, df3.Longitude,df3.Postcode):
    folium.Marker(
      location=[lat,lng],
      icon=None,
      popup=label,
).add_to(neighborhood)
map

This map suggests that postcodes are nor evenly scattered in neighborhoods of Toronto. The more close to Toronto there will be more postcodes being used. 